__#33. Вложенные классы__

Сам по себе экземпляр вложенного класса при создании экземлпяра базового не создается! Если мы хотим, чтоб он создавался - нужно прописать инициализацию в инициализаторе  

Во вложенных классах нельзя обращаться напрямую к атрибутам основного класса никак, кроме инициализатора вложенного класса, что не рекомендуется!

Подобное вложение используется для удобства, какого-то супер сакрального смысла в этом нет.

In [51]:
class Women: 
    title = 'объект для поля title'
    photo = 'объект класса для поля photo'
    ordering = 'объект класса для поля ordering'

    def __init__(self, user, psw):
        self._user = user 
        self._psw = psw 
        self.meta = self.Meta(user + "@" + psw)

    class Meta: 
        ordering = ['id']

        def __init__(self, access): 
            self._access = access

w =  Women('root', '12345')
print(w.__dict__)
print(w.meta.__dict__)

del Women

{'_user': 'root', '_psw': '12345', 'meta': <__main__.Women.Meta object at 0x000001A21F4C3770>}
{'_access': 'root@12345'}


__#34. Метаклассы. Объект type__

В пайтоне все классы эт обьекты, а объект, который позволяет создавать другие объекты - метакласс. 

В пайтоне метаклассом является объект type.

type(name, bases, dct) - создаеть динамически новый класс

"Метакласс - это глубокая магия, о которой 99% пользователей даже не нужно задумываться. Если вы думаете, нужно ли вам их использовать - вам не нужно. (люди, которым они реально нужны, точно знают, зачем они им, и не нуждаются в объяснениях почему)."

In [52]:
class Point: 
    MAX_COORD = 100 
    MIN_COORD = 0 

class B1: pass 
class B2: pass 

def method1(self):
    print(self.__dict__)

A = type('Point', (B1, B2), {'MAX_COORD': 100, 'MIN_COORD': 0, 'method1': lambda self: self.MAX_COORD}) # создали новый класс Point
pt = A()
pt.method1()

del Point, B1, B2

__#35. Пользовательские метаклассы. Параметр metaclass__

В пайтон можно конструировать свои собственные метаклассы

In [53]:
# def create_class_point(name, base, attrs): # описали простейший метакласс
#     attrs.update({'MAX_COORD': 100, 'MIN_COORD': 0})
#     return type(name, base, attrs)

class Meta(type):
    def __init__(cls, name, base, attrs):
        super().__init__(name, base, attrs)
        cls.MAX_COORD = 100
        cls.MIN_COORD = 0 

class Point(metaclass=Meta): 
    def get_coords(self):
        return (0, 0)
    
pt = Point()
print(pt.MAX_COORD)
print(pt.get_coords())

del Meta, Point

100
(0, 0)


__#36. Метаклассы в API ORM Django__

Django использует метаклассы для связи объектов с записями данных.


In [54]:
class Meta(type):
    def create_local_attrs(self, *args, **kwargs):
        for key, value in self.class_attrs.items():
            self.__dict__[key] = value 

    def __init__(cls, name, base, attrs):
        cls.class_attrs = attrs 
        cls.__init__ = Meta.create_local_attrs  

class Women(metaclass = Meta):
    title = 'заголовок'
    content = 'контент'
    photo = 'путь к фото'

w = Women()
print(w.__dict__)

del Meta, Women

{'__module__': '__main__', '__qualname__': 'Women', '__firstlineno__': 10, 'title': 'заголовок', 'content': 'контент', 'photo': 'путь к фото', '__static_attributes__': ()}


__#37. Введение в Python Data Classes (часть 1)__

Dataclasses позволяет сократить написания типовой писанины для описания классов.

При написании атрибутов в датаклассах их обязтельно нужно анотировать (писать тип данных какой str, int и т.п)

В датаклассах автоматически добавляется метод repr и инициализатор.

В датаклассах == или метод eq переопределены так, что при сравнении объектов сравниваются из переданные атрибуты, а не айдишники.

При объявлении параметров в датаклассах, все значения по умолчанию должны быть в конце, так как может возникнуть ошибка.

В датаклассах нельзя напрямую присаивать по умолчанию изменяемые объекты.

In [55]:
from dataclasses import dataclass, field

class Thing:
    def __init__(self, name , weight, price=0): 
        self.name = name 
        self.weight = weight 
        self.price = price 

    def __repr__(self): 
        return f"Thing: {self.__dict__}"
    
@dataclass 
class ThingData:
# в датаклассах можно прописать следующие свои магические методы: init, repr, eq
    name: str 
    weight: int 
    price: float = 0 # значение по умолчанию в конце
    dims: list = field(default_factory = list) # пример как можно изменяемый объект присвоить
    
t = Thing("Учебник по Python", 100, 1024)
td = ThingData("Учебник по Python", 100)
td2 = ThingData("Учебник по Python", 100)
td.dims.append(10)
print(td2.dims)

[]


In [56]:
from pprint import pprint # чтоб посмотреть че под капотом ThingData
pprint(ThingData.__dict__)
del Thing, ThingData

mappingproxy({'__annotations__': {'dims': <class 'list'>,
                                  'name': <class 'str'>,
                                  'price': <class 'float'>,
                                  'weight': <class 'int'>},
              '__dataclass_fields__': {'dims': Field(name='dims',type=<class 'list'>,default=<dataclasses._MISSING_TYPE object at 0x000001A21CEF9D30>,default_factory=<class 'list'>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({}),kw_only=False,_field_type=_FIELD),
                                       'name': Field(name='name',type=<class 'str'>,default=<dataclasses._MISSING_TYPE object at 0x000001A21CEF9D30>,default_factory=<dataclasses._MISSING_TYPE object at 0x000001A21CEF9D30>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({}),kw_only=False,_field_type=_FIELD),
                                       'price': Field(name='price',type=<class 'float'>,default=0,default_factory=<dataclasses._MISSING_TYPE object at 0x0

__#38. Введение в Python Data Classes (часть 2)__

 Функция field() предоставляет богатый функционал по управлению объявляемых атрибутов в Data Classes. Мы увидели, как работают два ее параметра: init и default_factory. Довольно часто можно встретить использование еще трех:

repr – булевое значение True/False указывает использовать ли атрибут в магическом методе __repr__() (по умолчанию True);

compare – булевое значение True/False указывает использовать ли атрибут при сравнении объектов (по умолчанию True);

default – значение по умолчанию (начальное значение).

Все атрибуты InitVar сразу попадают в постинит

До сих пор мы с вами использовали декоратор dataclass с параметрами по умолчанию. Однако ему можно передавать различные аргументы и управлять процессом формирования итогового класса. Вот основные параметры, которые принимает декоратор dataclass.

init = [True | False]

Принимает булево значение, по умолчанию True. Если значение True, то в классе объявляется инициализатор, иначе – не объявляется.

repr = [True | False]

Принимает булево значение, по умолчанию True. Если значение True, то в классе объявляется магический метод __repr__(), иначе – не объявляется.

eq = [True | False]

Принимает булево значение, по умолчанию True. Если значение True, то в классе объявляется магический метод __eq__(), иначе – не объявляется.

order = [True | False]

Принимает булево значение, по умолчанию False. Если значение True, то в классе объявляются магические методы для операций сравнения <; <=; >; >=, иначе – не объявляются.

unsafe_hash = [True | False]

Влияет на формирование магического метода __hash__()

frozen = [True | False]

Принимает булево значение, по умолчанию False. Если значение True, то атрибуты объектов класса становятся неизменными (можно только проинициализировать один раз в инициализаторе).

slots = [True | False]

Принимает булево значение, по умолчанию False. Если значение True, то атрибуты объявляются в коллекции __slots__.

In [57]:
from dataclasses import InitVar # чтоб можно было делать условия при вычислении выражений в датаклассах

class Vector3D:
    def __init__(self, x: int, y: int, z: int, calc_len: bool = True):
        self.x = x
        self.y = y
        self.z = z
        self.length = (x * x + y * y + z * z) ** 0.5 if calc_len else 0

@dataclass
class V3D:
    x: int = field(repr=False)
    y: int
    z: int = field(compare=False)
    calc_len: InitVar[bool] = True
    length: float = field(init=False, compare=False, default=0) # говорит, что атрибут length не следует добавлять, как парметр в инициализатор

    def __post_init__(self, calc_len: bool):
        if calc_len:    
            self.length = (self.x * self.x + self.y * self.y + self.z * self.z) ** 0.5


v = V3D(1, 2, 3)
v2 = V3D(1,2,5,False)
print(v)
print(v2)

V3D(y=2, z=3, length=3.7416573867739413)
V3D(y=2, z=5, length=0)


__#39. Python Data Classes при наследовании__

Когда формируется базовый класс и дочерний класс и в инициализаторе вызывается пост_инит, пост_инит ищется сначала в дочернем классе, а уже потом в базовом.

In [58]:
from dataclasses import dataclass, field, InitVar
from typing import Any
 
class GoodsMethodsFactory:
    @staticmethod
    def get_init_measure():
        return [0, 0, 0]
   
@dataclass
class Goods:
    current_uid = 0 # так как он не аннотирован - инициализатор его пропустит
    uid: int = field(init=False)
    price: Any = None
    weight: Any = None

    def __post_init__(self): 
        print("Goods: post_init")
        Goods.current_uid += 1
        self.uid = Goods.current_uid

@dataclass
class Book(Goods): 
    title: str = ""
    author: str = ""
    price: float = 0 
    weight: int | float = 0
    measure: list = field(default_factory=GoodsMethodsFactory.get_init_measure)

    def __post_init__(self):
        super().__post_init__()
        print("Book: post_init")    

b = Book(1000, 100, "Книга", "Автор")
print(b)

Goods: post_init
Book: post_init
Book(uid=1, price=1000, weight=100, title='Книга', author='Автор', measure=[0, 0, 0])


Еще можно создать новый датакласс, используя функцию make_dataclass()

Она имеет следующие наборы параметров:

dataclasses.make_dataclass(cls_name, fields, *, bases=(), namespace=None, init=True, repr=True, eq=True, order=False, unsafe_hash=False, frozen=False, match_args=True, kw_only=False, slots=False, weakref_slot=False)

Основные из них следующие:

cls_name – название нового класса (в виде строки);

fields – поля (локальные атрибуты) объектов класса;

* - произвольный набор позиционных аргументов;

bases – список базовых классов;

namespace – словарь для определения атрибутов самого класса (например, так можно объявлять методы класса).

Обычно, функцию make_dataclass() используют, если требуется сформировать класс данных в процессе работы программы. Если же нам нужно обычное объявление (а это наиболее частая ситуация), то гораздо удобнее использовать декоратор dataclass.

In [59]:
from dataclasses import make_dataclass

CarData = make_dataclass("CarData", [("model", str),
                                     "max_speed",
                                     ("price", float, field(default=0))],
                         namespace={'get_max_speed': lambda self: self.max_speed})

c = CarData("BMW", 256, 4096)
print(c)
print(c.get_max_speed())

CarData(model='BMW', max_speed=256, price=4096)
256
